In [1]:
import sys
sys.path.append('../')
import wrangle

import nlp

In [2]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

df, data_dict = wrangle.wrangle_data()

In [4]:
big_df = pd.read_csv('topics.csv', index_col=False)

In [5]:
big_df.drop('Unnamed: 0', axis=1,inplace=True)

In [6]:
mod_exp = big_df[big_df.persona_id == 4]

In [7]:
mod_exp.shape

(181, 31)

In [13]:
mod_exp[mod_exp.recommendations_topic_id == 'event, opportunity'].sort_values(by='recommendations_probability', ascending=False)

,prim_ind_text,prim_ind_topic_id,prim_ind_probability,types_res_text,types_res_topic_id,types_res_probability,future_res_text,future_res_topic_id,future_res_probability,research_educ_text,research_educ_topic_id,research_educ_probability,how_pick_events_text,how_pick_events_topic_id,how_pick_events_probability,events_attend_recent_text,events_attend_recent_topic_id,events_attend_recent_probability,ideal_topics_text,ideal_topics_topic_id,ideal_topics_probability,ideal_attendees_text,ideal_attendees_topic_id,ideal_attendees_probability,recommendations_text,recommendations_topic_id,recommendations_probability,big_answer,persona_id,polarity,subjectivity
681,Information,tech,0.562500,"Quantitative by metrics and A/B results, quali...",qual/quant,0.485996,Generative methods,focus group,0.708225,Earned master's degree and undergraduate degre...,"masters, bachelors",0.967766,"Timing, cost, funding availability, and if the...",pay,0.947758,"Conferences, meetups from local professional orgs","toronto, strive",0.904352,"Strategy, roadmapping, stakeholder management,...",quant/qual/data,0.901351,Researchers who are doing cool things outside ...,"experience, jared spool",0.943222,Make sure the presenters provide actionable ta...,"event, opportunity",0.979146,"Yes, I was taught how to conduct researchEarne...",4,0.141667,0.373119
434,"Architecture, Engineering, & Construction (AEC...",education,0.829510,"Mostly qualitative, but more & more quantititi...",ngram,0.927031,We are trying to build more muscles around A/B...,market research,0.874900,NaN,"masters, bachelors",0.166667,If the topic is of interest to me I consider i...,pay,0.966310,"CHI, UXPA & NNG events","toronto, strive",0.856766,"Statistics (the good, the bad, and the ugly); ...",ops/ai,0.842157,"Genevieve Bell, Nir Eyal, Indi Young, Shantell...","sam ladner, erika hall",0.940987,Think outside the typical conference box. Allo...,"event, opportunity",0.978025,"No, I was not taught how to conduct researchn/...",4,0.200901,0.469369
223,healthcare,healthcare,0.562500,"Field visits/contextual inquiry, resonance tes...",ngram,0.435263,Pretty happy with our current stable of techni...,market research,0.874910,Masters in human factors. First job out of gra...,"masters, bachelors",0.575659,New topics that give me the impression I'll ac...,"price, location",0.567955,"None, I would rather focus on design or produc...","urca, uxpa",0.591790,How to democratize research and support all di...,"new_method, mixed_method",0.866981,Honestly not sure. Haven't given it much thoug...,"experience, jared spool",0.872141,I do and direct a LOT of research and so do th...,"event, opportunity",0.975952,"Yes, I was taught how to conduct researchMaste...",4,0.115990,0.413440
717,Network Security,education,0.708262,"Usability Testing, also UX interviews (a hybri...",ngram,0.746435,"Diary study, contextual inquiry",card sort,0.644519,I have a master in information with an HCI focus,"class, running study",0.617523,"For a local event, I first look at the topic t...","value, reputation",0.978355,Wall of Knowledge,"london, user_research",0.142857,Some new tools and methods in the research fie...,quant/qual/data,0.772868,NaN,"industry, team, product",0.250000,JUST DO IT!\n\nEven though I have been conduct...,"event, opportunity",0.972963,"Yes, I was taught how to conduct researchI hav...",4,0.311718,0.443605
565,SaaS,commerce/consulting,0.562365,"Qualitative, quantitative, and mixed methods (...",testing,0.953910,Possibly investing more in quantitative resear...,"moderate, unmoderate",0.874889,humanities PhD (with teaching component) focus...,"experimental, psychology",0.880338,Looking for depth of UX research topics (not j...,"value, reputation",0.569944,NaN,"london, user_research",0.142857,"Growing a team, advanced methods or analysis t...","new_method, mixed_method",0.944396,NaN,"industry, team, product",0.250000,Conference funding isn't necessarily guarantee...,"event, opportunity",0.972859,"Yes, I was taught how 

In [8]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference', 'yes', 've', 'ha']

stopWords = nlp.set_stop_words(stop_words)

# Moderately Experienced

## Sentiment

In [11]:
mod_exp.big_answer.apply(nlp.find_polarity).mean()

0.22334931824328036

In [12]:
mod_exp.big_answer.apply(nlp.find_subjectivity).mean()

0.4380907988054033

## Total Keywords

In [15]:
nlp.show_column_keywords(mod_exp.big_answer, max_df=.8, stop_words=stopWords, ngram_range=(1,3))

['researchnatravel',
 'cost',
 'time',
 'scheduling',
 'need',
 'fit',
 'quality',
 'event',
 'specific',
 'topic',
 'covered',
 'material',
 'experience',
 'versus',
 'design',
 'run',
 'expectation',
 'great',
 'balance',
 'inspiration']

## What topics would they be most attracted to at a conference about research?

q21 ideal_topics

In [18]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference', 'good', 'best', 'self', 'report']

In [19]:
stopWords = nlp.set_stop_words(stop_words)

### keywords

In [23]:
#words
nlp.show_column_keywords(mod_exp.ideal_topics_text, max_df=.5, stop_words=stopWords, ngram_range=(1,3))

['mixed',
 'method',
 'approach',
 'artifact',
 'design',
 'presentation',
 'mixed method',
 'method approach',
 'question',
 'data',
 'influencing',
 'leadership',
 'management',
 'global',
 'leading',
 'team',
 'way',
 'plan',
 'support',
 'skill']

In [22]:
#bi-grams
nlp.show_column_keywords(mod_exp.ideal_topics_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

['mixed method',
 'method approach',
 'leading team',
 'product development',
 'large organization',
 'ops case',
 'case study',
 'ops case study',
 'product team',
 'method analysis',
 'analysis method',
 'study practice',
 'case study practice',
 'business strategy',
 'new method',
 'career path',
 'design product',
 'structure team',
 'working product',
 'agile product']

### topics

In [9]:
mod_exp[mod_exp.ideal_topics_text.notnull()].ideal_topics_topic_id.value_counts()

new_method, mixed_method    48
quant/qual/data             40
case_study                  29
ops/ai                      28
Name: ideal_topics_topic_id, dtype: int64

## Who would they expect to see at a conference about research?

q22 Ideal Attendees

In [11]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference']

In [12]:
stopWords = nlp.set_stop_words(stop_words)

In [25]:
#terms
nlp.show_column_keywords(mod_exp.ideal_attendees_text, max_df=.5, stop_words=stopWords)

['steve',
 'portigal',
 'steve portigal',
 'genevieve',
 'young',
 'graduate',
 'importantly',
 'client',
 'experience',
 'startup',
 'consumer',
 'manager',
 'researcher',
 'present',
 'case',
 'study',
 'case study',
 'jared',
 'spool',
 'team']

In [26]:
#bigrams
nlp.show_column_keywords(mod_exp.ideal_attendees_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

['steve portigal',
 'case study',
 'jared spool',
 'spool steve',
 'organisation google',
 'google facebook',
 'jared spool steve',
 'spool steve portigal',
 'small large',
 'public sector',
 'queer color',
 'present perspective',
 'design team',
 'leader field',
 'sam ladner',
 'leisa reichelt',
 'reichelt erika',
 'erika hall',
 'leisa reichelt erika',
 'reichelt erika hall']

In [10]:
mod_exp[mod_exp.ideal_attendees_text.notnull()].ideal_attendees_topic_id.value_counts()

industry, team, product    42
indi young                 34
sam ladner, erika hall     27
experience, jared spool    25
Name: ideal_attendees_topic_id, dtype: int64

## What advice do they have for the Rosenfeld Media team in pursuing a conference?
q23 recommendations

In [28]:
#words
nlp.show_column_keywords(mod_exp.recommendations_text, max_df=.9, stop_words=stopWords, ngram_range=(1,7))

['read',
 'book',
 'learn',
 'lecture',
 'hand',
 'help',
 'run',
 'workshop',
 'career',
 'sure',
 'design',
 'difficult',
 'business',
 'approach',
 'try',
 'teach',
 'case',
 'away',
 'world',
 'fun']

In [29]:
#ngrams
nlp.show_column_keywords(mod_exp.recommendations_text, max_df=.9, stop_words=stopWords, ngram_range=(2,7))

['interesting way',
 'consider bringing',
 'ticket price',
 'price point',
 'product manager',
 'practical application',
 'industry background',
 'hand activity',
 'speaker new',
 'great idea',
 'thought leader',
 'talk workshop',
 'person hear',
 'informed consent',
 'participant time',
 'north america']

In [11]:
mod_exp[mod_exp.recommendations_text.notnull()].recommendations_topic_id.value_counts()

speaker, industry      36
event, opportunity     29
good, know             26
group, career, city    25
field, survery         17
Name: recommendations_topic_id, dtype: int64

## Top Documents per Topic

In [31]:
doc_term_matrix, count_vect = nlp.create_wordcount_matrix(mod_exp.recommendations_text, max_df=.3, ngram=(1,3), stop_words=stopWords)

LDA = LatentDirichletAllocation(n_components=4, random_state=42)

LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
recommendations_dict = {0 : 'speaker, industry', 
                        1 : 'event, opportunity', 
                        2 : 'good, know', 
                        3 : 'field, survery', 
                        4 : 'group, career, city'}

In [32]:
nlp.find_top_documents_per_topic(LDA.transform(doc_term_matrix), mod_exp.recommendations_text, 5)

Top 5 Documents for Topic 0: 

Document 1
I realize this is challenging but cost matters so try to keep it low. In some cases, I've paid for things myself because the company hasn't paid for it. I wish more event planners would consider regular everyday job blow who can't pay thousands to go to a conference, especially if they need to travel too. 

Document 2
Make it a mix of workshops and talks and try to avoid the tired "advocate for the user", "be empathetic" user research rallying cries of yore. That stuff is boring and meaningless. Explore research ethics and design justice, informed consent, and what it means to scale user research in organisations and why it's difficult.

Document 3
Have a well-written code of conduct.
Share materials (slides, talks) afterward as much as possible. Having access to these allows researchers to socialize what we learned within our orgs.
Cater to more experienced researchers - there are already so many places to learn methods and basics but very few